<h2 style='color:purple' align='center'>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash</h2>

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/titanic.csv")
df.head()

PassengerId                                               Name  Pclass  \
0            1                            Braund, Mr. Owen Harris       3   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...       1   
2            3                             Heikkinen, Miss. Laina       3   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)       1   
4            5                           Allen, Mr. William Henry       3   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  \
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S   
1  female  38.0      1      0          PC 17599  71.2833   C85        C   
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S   
3  female  35.0      1      0            113803  53.1000  C123        S   
4    male  35.0      0      0            373450   8.0500   NaN        S   

   Survived  
0         0  
1         1  
2         1  
3         1  
4         0

In [6]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

Pclass     Sex   Age     Fare  Survived
0       3    male  22.0   7.2500         0
1       1  female  38.0  71.2833         1
2       3  female  26.0   7.9250         1
3       1  female  35.0  53.1000         1
4       3    male  35.0   8.0500         0

In [7]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [8]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [9]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

female   male
0   False   True
1    True  False
2    True  False

In [10]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3) #head shows first 5 rows

Pclass     Sex   Age     Fare  female   male
0       3    male  22.0   7.2500   False   True
1       1  female  38.0  71.2833    True  False
2       3  female  26.0   7.9250    True  False

**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [11]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

Pclass   Age     Fare  female
0       3  22.0   7.2500   False
1       1  38.0  71.2833    True
2       3  26.0   7.9250    True

In [12]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [13]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [24]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head(6)

Pclass        Age     Fare  female
0       3  22.000000   7.2500   False
1       1  38.000000  71.2833    True
2       3  26.000000   7.9250    True
3       1  35.000000  53.1000    True
4       3  35.000000   8.0500   False
5       3  29.699118   8.4583   False

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3) #size=0.3 means that 30% data is used for testing

In [16]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [17]:
model.fit(X_train,y_train)

GaussianNB()

In [18]:
model.score(X_test,y_test)

0.7611940298507462

In [19]:
X_test[0:10]

Pclass        Age      Fare  female
302       3  19.000000    0.0000   False
713       3  29.000000    9.4833   False
307       1  17.000000  108.9000    True
14        3  14.000000    7.8542    True
333       3  16.000000   18.0000   False
820       1  52.000000   93.5000    True
100       3  28.000000    7.8958    True
280       3  65.000000    7.7500   False
233       3   5.000000   31.3875    True
29        3  29.699118    7.8958   False

In [20]:
y_test[0:10]

302    0
713    0
307    1
14     0
333    0
820    1
100    0
280    0
233    1
29     0
Name: Survived, dtype: int64

In [21]:
model.predict(X_test[0:10])

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0])

In [22]:
model.predict_proba(X_test[:10])

array([[0.95392724, 0.04607276],
       [0.96285958, 0.03714042],
       [0.00154802, 0.99845198],
       [0.33802291, 0.66197709],
       [0.95325773, 0.04674227],
       [0.00486677, 0.99513323],
       [0.40351517, 0.59648483],
       [0.94261489, 0.05738511],
       [0.25013514, 0.74986486],
       [0.96290266, 0.03709734]])

**Calculate the score using cross validation**

In [23]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.808     , 0.736     , 0.728     , 0.76612903, 0.86290323])